In [3]:
import psycopg2
import json
import codecs
from ast import literal_eval

In [4]:
def check_value(jsonkey):
    try: 
        return reviewjson[jsonkey]
    except KeyError:
        return ''

<h1> connect to db and create table </h1>

In [5]:
conn = psycopg2.connect("host=localhost port=5432 dbname=amazon user=postgres password=darkmatter")
cur = conn.cursor()

<h1> Create a Table with json as the column </h1> 

In [6]:
#Reviews db ... make seperate tables
def create_review_table(table_name):
    cur.execute("CREATE TABLE reviews_{:s} (id serial PRIMARY KEY,\
        catagory text,\
        summary text,\
        helpful int[],\
        reviewerID text,\
        reviewerName text,\
        overall float,\
        asin text,\
        reviewTime Timestamp,\
        reviewText text,\
        unixReviewTime int\
        );".format(table_name))
    conn.commit() #submit change to db

<h1> read in 20 GB file one line at a time and write to the db, from reviews data file </h1>

In [7]:
#note the catagory = file name (amazon product catigory)   
file_list=!ls /home/ubuntu/amazon/Reviews
for catagory_file in file_list[16:]:
    table_name=catagory_file.split('_5')[0]
    create_review_table(table_name)
    with open('/home/ubuntu/amazon/Reviews/'+catagory_file) as f:
        for line in f:
            reviewjson=json.loads(line)

            try:
                asin=reviewjson['asin']
            except KeyError:
                asin=''
            try:
                helpful= reviewjson['helpful']
            except KeyError:
                helpful=[0,0]

            summary=check_value('summary')
            reviewerID= check_value('reviewerID')
            reviewerName= check_value('reviewerName')
            overall= check_value('overall')
            #asin reviewjson['asin']
            reviewTime= check_value('reviewTime')
            reviewText= check_value('reviewText')
            unixReviewTime= check_value('unixReviewTime')

            #insert into postgres
            cur.execute("INSERT INTO reviews_{:s} (\
            catagory,\
            summary,\
            helpful,\
            reviewerID,\
            reviewerName,\
            overall,\
            asin,\
            reviewTime,\
            reviewText,\
            unixReviewTime\
            )\
            VALUES (\
            %s,%s,%s,%s,%s,%s,%s,%s,%s,%s\
            )".format(table_name),(catagory_file,summary,helpful,reviewerID,reviewerName,overall,asin,reviewTime,reviewText,unixReviewTime )\
            )
            conn.commit() #submit change to db
 

In [2]:
file_list=!ls /home/ubuntu/amazon/Reviews
file_list[15:]

['Movies_and_TV_5.json',
 'Musical_Instruments_5-2.json',
 'Office_Products_5.json',
 'Patio_Lawn_and_Garden_5.json',
 'Pet_Supplies_5.json',
 'Sports_and_Outdoors_5.json',
 'Tools_and_Home_Improvement_5.json',
 'Toys_and_Games_5.json',
 'Video_Games_5.json']